In [28]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras.backend as K
from keras.preprocessing import sequence
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, Input, BatchNormalization, Conv1D, Multiply, Activation, MaxPooling1D
from keras.regularizers import l2
from keras.optimizers import Adam, SGD

## Below path hardcoded. TODO: Change this
prefix_path = '..'

labels = pd.read_csv(prefix_path + '/train_kaggle.csv')

print('Labels', labels.describe())


X = []
y = []

max_len = 350
batch_size = 512





Labels                  Id         label
count  30336.000000  30336.000000
mean   15167.500000      0.077664
std     8757.393219      0.267646
min        0.000000      0.000000
25%     7583.750000      0.000000
50%    15167.500000      0.000000
75%    22751.250000      0.000000
max    30335.000000      1.000000


In [29]:
## For each sample in the file
for index, train_label in labels.iterrows():
    label = train_label['label']

    ## zeros_array used to keep the maximum number of sequences constant to max_len
    zeros_array = np.zeros((max_len, 40))

    ## features is a (N, 40) matrix
    features = np.load(prefix_path + '/train/train/' + str(train_label['Id']) + '.npy')
    
    ## We add it to zeros_array to make all samples as (400, 40) matrix
    zeros_array[0:len(features)] = features


    ## For each feature, we find average of all values and replace all NaN with that value
    for feature in range(40):
        average_value = np.average(zeros_array[:feature][np.nan_to_num(zeros_array[:feature]) != 0])
        zeros_array[:feature] = np.nan_to_num(zeros_array[:feature], average_value)

    X.append(zeros_array)
    y.append(label)

X = np.array(X)
y = np.array(y)

print('X Shape', X.shape)
print('y shape', y.shape)


X Shape (30336, 350, 40)
y shape (30336,)


In [30]:

sample_array = [[[1, 2, 3], [4, 5, 6]], [[1, 2, 3], [4, 5, 6]]]
sample_array = np.array(sample_array)
print(sample_array.shape)
print('Total average', np.mean(sample_array))
print('Feats average', np.mean(sample_array, axis=(0, 1)))

(2, 2, 3)
Total average 3.5
Feats average [2.5 3.5 4.5]


In [31]:
num_samples, num_sequences, num_features = X.shape[0], X.shape[1], X.shape[2]
print("{} Samples, {} sequential samples, {} features".format(num_samples, num_sequences, num_features))
feature_wise_average = np.mean(X, axis=(0, 1))
assert feature_wise_average.shape[0] == num_features
print("Feature-wise average")
print(feature_wise_average)

for feature in range(num_features):
    feature_values = X[:, :, feature]
    #print('Feature {} ; NaN values: {} out of {}'.format(feature, np.count_nonzero(feature_values), feature_values.shape[0] * feature_values.shape[1]))
    print('Feature {} ; NaN values: {} out of {}'.format(feature, np.count_nonzero(~np.isnan(feature_values)), feature_values.shape[0] * feature_values.shape[1]))

30336 Samples, 350 sequential samples, 40 features
Feature-wise average
[        nan         nan  2.99179363         nan         nan         nan
         nan         nan         nan         nan         nan  6.8681197
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan  0.06232454         nan -5.93850188
         nan         nan         nan         nan]
Feature 0 ; NaN values: 10525041 out of 10617600
Feature 1 ; NaN values: 10432727 out of 10617600
Feature 2 ; NaN values: 10617600 out of 10617600
Feature 3 ; NaN values: 10587396 out of 10617600
Feature 4 ; NaN values: 10423339 out of 10617600
Feature 5 ; NaN values: 10425357 out of 10617600
Feature 6 ; NaN values: 10432825 out of 10617600
Feature 7 ; NaN values: 10525041 out of 10617600
Feature 8 ; NaN values: 10436829 out of 10617600


In [38]:
X_2d = np.sum(X, axis=1)
print(X_2d.shape)
print(X_2d)

(30336, 40)
[[    nan     nan 1485.   ... 1241.       nan     nan]
 [  23.      2.5   276.   ...  270.5     0.    216.99]
 [   0.       nan 1176.   ...     nan     nan     nan]
 ...
 [   0.      7.4   630.   ...  701.5    31.4   261.7 ]
 [    nan     nan 1081.   ...  675.       nan     nan]
 [   0.      0.     36.   ...  125.      0.     71.9 ]]
